In [1]:
from pandas import DataFrame, merge, concat, read_csv
from pickle import dump, load

In [2]:
DataRaw = merge( read_csv( 'train_transaction.csv' ), read_csv( 'train_identity.csv' ), on = 'TransactionID', how = 'left' )
DataRaw.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [3]:
DataCat = DataRaw.select_dtypes( include = ['object'] )
DataRaw.drop( columns = DataCat.columns, inplace = True )
for col in DataRaw.columns:
    DataRaw[ col ] = DataRaw[ col ].fillna( DataRaw[col].mean() )

In [4]:
with open( 'DataRaw.pkl', 'wb' ) as f:
    dump( DataRaw, f )
    dump( DataCat, f )

In [5]:
import pandas as pd
import sklearn as sk
import seaborn as sb
import matplotlib.pyplot as pp
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
encoder = OneHotEncoder( sparse_output = False ).set_output( transform = 'pandas' )

WindSpeed: pd.DataFrame = pd.read_csv( 'data_treino_dv_df_2000_2010.csv' )
WindSpeed.rename( columns = { 'HORA (UTC)': 'Hora', 
    'VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))': 'Direccion',
    'VENTO, VELOCIDADE HORARIA (m/s)': 'Velocidad',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'Humedad Max',
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'Humedad Min',
    'TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C)': 'Temp Max',
    'TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C)': 'Temp Min',
    'UMIDADE RELATIVA DO AR, HORARIA (%)': 'Humedad Rel',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'Presion',
    'PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)': 'Precipitacion',
    'VENTO, RAJADA MAXIMA (m/s)': 'Rafaga',
    'PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'Presion Max',
    'PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'Presion Min'
}, inplace = True )

ws = WindSpeed

hour = encoder.fit_transform(WindSpeed[['Hora']])
encoded = pd.DataFrame(hour, columns = encoder.get_feature_names_out(['Hora']))
WindSpeed = pd.concat([WindSpeed, encoded], axis=1).drop(columns=['Hora'])

In [6]:
X = WindSpeed.drop( columns=['Velocidad'] )
y = WindSpeed['Velocidad']

In [7]:
X

,Direccion,Humedad Max,Humedad Min,Temp Max,Temp Min,Humedad Rel,Presion,Precipitacion,Rafaga,Presion Max,...,Hora_14:00,Hora_15:00,Hora_16:00,Hora_17:00,Hora_18:00,Hora_19:00,Hora_20:00,Hora_21:00,Hora_22:00,Hora_23:00
0,0.809017,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.965926,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.891007,56.0,50.0,25.5,24.3,51.0,888.1,0.0,4.9,888.4,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.848048,52.0,44.0,27.4,25.0,44.0,887.4,0.0,5.8,888.1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.224951,50.0,43.0,27.1,25.5,46.0,886.5,0.0,5.8,887.4,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87688,-0.615661,83.0,78.0,21.8,21.1,80.0,879.1,0.0,12.3,879.8,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
87689,-0.469472,84.0,79.0,21.7,21.0,84.0,879.2,0.0,9.9,879.2,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
87690,-0.484810,86.0,82.0,21.2,20.6,86.0,879.7,0.0,8.9,879.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
87691,-0.484810,88.0,85.0,20.6,20.2,88.0,880.5,0.0,8.0,880.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
y

0        1.8
1        2.7
2        2.0
3        2.5
4        2.4
        ... 
87688    5.6
87689    4.9
87690    4.5
87691    3.2
87692    2.3
Name: Velocidad, Length: 87693, dtype: float64

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
CVSplits

NameError: name 'CVSplits' is not defined

In [8]:
X.isnull().sum().sum()

0

Por partición se entiende: La iteración i+1 será a partir del segundo dia de la iteración anterior.

In [110]:
CVSplits = [
    [
        (
            list(range( 24*i, 24*i + k )),
            list(range( 24*i + k, 24*(i+1) + k ))
        ) 
        for i in range( len(X)//24 - 168) 
    ]
    for k in [ 168, 336, 504, 672 ]
]

In [111]:
CVSplits[3][-1][1]

[84288,
 84289,
 84290,
 84291,
 84292,
 84293,
 84294,
 84295,
 84296,
 84297,
 84298,
 84299,
 84300,
 84301,
 84302,
 84303,
 84304,
 84305,
 84306,
 84307,
 84308,
 84309,
 84310,
 84311]

In [112]:
with open( 'Windspeed.pkl', 'wb' ) as f:
    dump( (X_train, X_test), f )
    dump( (y_train, y_test), f )
    dump( CVSplits, f )

In [93]:

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score, make_scorer

from numpy import mean

linearScores = pd.DataFrame( columns = ['T = 7', 'T = 14', 'T = 21'] )

for CVSplit in CVSplits:
    scores = cross_val_score( LinearRegression(), X, y, cv = CVSplit, n_jobs=5 )